<a href="https://colab.research.google.com/github/kleczekr/tolkenizer/blob/master/violence8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline 

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # needed for waffle Charts
# import seaborn as sns
import numpy as np
import folium

In [2]:
# mpl.style.use(['ggplot'])

In [3]:
# This cell is meant to accommodate the Google Colab way of dealing with reading 
# files from Google Drive; feel free to ignore it if you are running the notebook
# on your local machine
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_excel('/content/drive/My Drive/ethnic_violence/globalterrorismdb_0919dist.xlsx')

In [5]:
condition = (df.country_txt.isin(['India', 'Pakistan', 'Bangladesh', 'Bhutan', 'Sri Lanka']) & (df.iyear == 1991))
df_sa_91 = df[condition]

In [6]:
pd.set_option('display.max_columns', 150)
df_sa_91 = df_sa_91.fillna(0)
df_sa_91.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
44984,199101020012,1991,1,2,0,0,0,186,Sri Lanka,6,South Asia,Eastern,Elephant Pass,9.518485,80.406057,1.0,0,0,0,1,1,0,1,1.0,Insurgency/Guerilla Action,0,1,0,2,Armed Assault,0.0,0,0.0,0,4,Military,27.0,Military Barracks/Base/Headquarters/Checkpost,Military,Camp,186.0,Sri Lanka,0.0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0,0,0.0,0,Liberation Tigers of Tamil Eelam (LTTE),0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,5,Firearms,2.0,Automatic or Semi-Automatic Rifle,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,Automatic firearm,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0,0,0,PGIS,0,0,0,0,0
44985,199101020013,1991,1,2,0,0,0,186,Sri Lanka,6,South Asia,Northern,Kankesanturai,9.815254,80.037053,1.0,0,0,0,1,1,0,1,1.0,Insurgency/Guerilla Action,0,1,0,3,Bombing/Explosion,0.0,0,0.0,0,4,Military,29.0,Military Unit/Patrol/Convoy,Military,Detachment,186.0,Sri Lanka,0.0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0,0,0.0,0,Liberation Tigers of Tamil Eelam (LTTE),0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,6,Explosives,11.0,"Projectile (rockets, mortars, RPGs, etc.)",0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,Mortars,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0,0,0,PGIS,0,0,0,0,0
45062,199101070006,1991,1,7,0,0,0,92,India,6,South Asia,West Bengal,Purulia,23.332078,86.365208,1.0,0,0,0,1,1,1,-9,0.0,0,0,1,0,2,Armed Assault,0.0,0,0.0,0,3,Police,25.0,Police Security Forces/Officers,Police,Unit,92.0,India,0.0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0,0,0.0,0,Sikh Extremists,Militants,0,0,0,0,0,0.0,0.0,0.0,0,3.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,5,Firearms,2.0,Automatic or Semi-Automatic Rifle,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,Automatic firearm,0.0,0.0,0.0,3.0,0.0,0.0,1,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0,0,0,PGIS,0,0,0,0,0
45070,199101080008,1991,1,8,0,0,0,92,India,6,South Asia,Punjab,Ludhiana,30.898024,75.860077,1.0,0,0,0,1,1,1,0,0.0,0,0,1,0,6,Hostage Taking (Kidnapping),0.0,0,0.0,0,14,Private Citizens & Property,69.0,Religion Identified,0,Hindus at Railway Station,92.0,India,0.0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0,0,0,0.0,0,Sikh Extremists,Militants,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,5,Firearms,2.0,Automatic or Semi-Automatic Rifle,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,Automatic firearm,12.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,1.0,14.0,0.0,0.0,0.0,0,India,0.0,0.0,0.0,0.0,0.0,0,7.0,Unknown,2.0,0,0,0,0,PGIS,0,0,0,0,0
45074,199101090004,1991,1,9,0,0,0,153,Pakistan,6,South Asia,Punjab,Unknown,31.090612,72.35

In [7]:
df_sa_91.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646 entries, 44984 to 49640
Columns: 135 entries, eventid to related
dtypes: float64(53), int64(45), object(37)
memory usage: 686.4+ KB


In [8]:
# India latitude and longitude values
latitude = 20.5937
longitude = 78.9629

In [9]:
# create map and display it
sa_map = folium.Map(location=[latitude, longitude], zoom_start=4)

# display the map of San Francisco
sa_map

In [10]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_sa_91.latitude, df_sa_91.longitude):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=2, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(df_sa_91.latitude)
longitudes = list(df_sa_91.longitude)
labels = list(df_sa_91.attacktype1_txt)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(sa_map)    
    
# add incidents to map
sa_map.add_child(incidents)

In [11]:
# create map and display it
sa_map = folium.Map(location=[latitude, longitude], zoom_start=4)

# loop through the 100 crimes and add each to the map
for lat, lng, label in zip(df_sa_91.latitude, df_sa_91.longitude, df_sa_91.attacktype1_txt):
    folium.CircleMarker(
        [lat, lng],
        radius=4, # define how big you want the circle markers to be
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sa_map)

# show map
sa_map

In [12]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sa = folium.Map(location = [latitude, longitude], zoom_start = 4)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sa_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_sa_91.latitude, df_sa_91.longitude, df_sa_91.attacktype1_txt):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
sa_map